<a href="https://colab.research.google.com/github/T-Sunm/Learn-Data-Cleaning-in-kaggle/blob/main/Exercise_Inconsistent_Data_Entry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'pakistan-intellectual-capital:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F819513%2F1402182%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240909%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240909T154135Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8e3b8793228f864a3f7c68e8937bed00faf84565e29a97be632f0b4697192c0ec4e82c75c6a106f61c2f3c42d19ddf332d7b245923b36c0ec88f29906e00d233e4acf5c6d6b0addd4c4b8c10a4ce05326ae2458dd09f8bc7fdacf78a03f2b77bee1baf523fb6db7181b66cdc6d472b9edf9a3e37c6892be4f0cc641a8ea9e1ca861150cb69b7659e17ee64c5441f86db707257cb89a7a4df3d720caaab3b178d584fc0bc7de3f8a87e3ac1d972c528a22c4675c4a9a473bc8cac43fc3e66e2aa00abc698cb7bbd25e993fb6673ad65528eeb2cfe6d7441f154f9028a2d65bd52236c17198dc844dd8a11b8f60d5492c9606af367fb1a3c9c329bdd8a20321385'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


**This notebook is an exercise in the [Data Cleaning](https://www.kaggle.com/learn/data-cleaning) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/inconsistent-data-entry).**

---


In this exercise, you'll apply what you learned in the **Inconsistent data entry** tutorial.

# Setup

The questions below will give you feedback on your work. Run the following cell to set up the feedback system.

In [ ]:
from learntools.core import binder
binder.bind(globals())
from learntools.data_cleaning.ex5 import *
print("Setup Complete")

Setup Complete


# Get our environment set up

The first thing we'll need to do is load in the libraries and dataset we'll be using.  We use the same dataset from the tutorial.

In [ ]:
# modules we'll use
import pandas as pd
import numpy as np

# helpful modules
import fuzzywuzzy
from fuzzywuzzy import process
import charset_normalizer

In [ ]:
with open("../input/pakistan-intellectual-capital/pakistan_intellectual_capital.csv", 'rb') as rawdata:
    result = charset_normalizer.detect(rawdata.read(1000000))
print(result)

{'encoding': 'utf-8', 'language': 'English', 'confidence': 1.0}


In [ ]:
# read in all our data
professors = pd.read_csv("../input/pakistan-intellectual-capital/pakistan_intellectual_capital.csv", encoding = 'utf-8')

# set seed for reproducibility
np.random.seed(0)

Next, we'll redo all of the work that we did in the tutorial.

In [ ]:
# convert to lower case
professors['Country'] = professors['Country'].str.lower()
# remove trailing white spaces
professors['Country'] = professors['Country'].str.strip()

# get the top 10 closest matches to "south korea"
countries = professors['Country'].unique()
matches = fuzzywuzzy.process.extract("south korea", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
print(matches)
def replace_matches_in_column(df, column, string_to_match, min_ratio = 47):
   column_to_match = df[column].unique()

   matches = fuzzywuzzy.process.extract(string_to_match, column_to_match, limit = 10, scorer = fuzzywuzzy.fuzz.token_sort_ratio)
   print(string_to_match,matches)
   # filter matches:
   matches_name = [match[0] for match in matches if match[1] >= min_ratio]

   # find name meet condition
   mask_cond = df[column].isin(matches_name)

   # replace string to match
   df.loc[mask_cond, column] = string_to_match
   print

replace_matches_in_column(df=professors, column='Country', string_to_match="south korea")
countries = professors['Country'].unique()

[('south korea', 100), ('southkorea', 48), ('saudi arabia', 43), ('norway', 35), ('ireland', 33), ('portugal', 32), ('singapore', 30), ('netherland', 29), ('usofa', 25), ('macau', 25)]
south korea [('south korea', 100), ('southkorea', 48), ('saudi arabia', 43), ('norway', 35), ('ireland', 33), ('portugal', 32), ('singapore', 30), ('netherland', 29), ('usofa', 25), ('macau', 25)]


# 1) Examine another column

Write code below to take a look at all the unique values in the "Graduated from" column.

In [ ]:
professors.head()

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,thailand,NaN,Software Engineering & DBMS,NaN
1,4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,thailand,NaN,DBMS,NaN
2,5,6,Dr. Junaid Baber,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,thailand,NaN,"Information processing, Multimedia mining",NaN
3,6,7,Dr. Maheen Bakhtyar,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,thailand,NaN,"NLP, Information Retrieval, Question Answering...",NaN
4,24,25,Samina Azim,Sardar Bahadur Khan Women's University,Computer Science,Balochistan,Lecturer,BS,Balochistan University of Information Technolo...,pakistan,2005.0,VLSI Electronics DLD Database,NaN


In [ ]:
professors["Graduated from"].unique()[0:5]

array(['Asian Institute of Technology',
       'Balochistan University of Information Technology, Engineering and Management Sciences',
       'University of Balochistan',
       "Sardar Bahadur Khan Women's University",
       'SRH Hochschule Heidelberg'], dtype=object)

Do you notice any inconsistencies in the data?  Can any of the inconsistencies in the data be fixed by removing white spaces at the beginning and end of cells?

Once you have answered these questions, run the code cell below to get credit for your work.

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 

There are inconsistencies that can be fixed by removing white spaces at the beginning and end of cells.  For instance, "University of Central Florida" and " University of Central Florida" both appear in the column.

In [ ]:
# Line below will give you a hint
#q1.hint()

# 2) Do some text pre-processing

Convert every entry in the "Graduated from" column in the `professors` DataFrame to remove white spaces at the beginning and end of cells.

In [ ]:
professors['Graduated from'].unique()[0:5]

array(['Asian Institute of Technology',
       'Balochistan University of Information Technology, Engineering and Management Sciences',
       'University of Balochistan',
       "Sardar Bahadur Khan Women's University",
       'SRH Hochschule Heidelberg'], dtype=object)

In [ ]:
professors['Graduated from'] = professors['Graduated from'].str.strip()
# Check your answer
q2.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
# q2.hint()
# q2.solution()

# 3) Continue working with countries

In the tutorial, we focused on cleaning up inconsistencies in the "Country" column.  Run the code cell below to view the list of unique values that we ended with.

In [ ]:
# get all the unique values in the 'City' column
countries = professors['Country'].unique()


# sort them alphabetically and then take a closer look
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea', 'spain',
       'sweden', 'thailand', 'turkey', 'uk', 'urbana', 'usa', 'usofa'],
      dtype=object)

Take another look at the "Country" column and see if there's any more data cleaning we need to do.

It looks like 'usa' and 'usofa' should be the same country.  Correct the "Country" column in the dataframe to replace 'usofa' with 'usa'.

**Use the most recent version of the DataFrame (with the whitespaces at the beginning and end of cells removed) from question 2.**

In [ ]:
matches = fuzzywuzzy.process.extract("usa", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('usa', 100),
 ('usofa', 75),
 ('austria', 60),
 ('australia', 50),
 ('spain', 50),
 ('urbana', 44),
 ('uk', 40),
 ('malaysia', 36),
 ('pakistan', 36),
 ('portugal', 36)]

In [ ]:
# valid_countries  = ['Australia', 'Canada', 'China', 'Finland', 'France',
#                    'Germany', 'Greece', 'Hong Kong', 'Ireland', 'Italy', 'Japan',
#                    'Macau', 'Malaysia', 'Mauritius', 'Netherlands', 'New Zealand',
#                    'Norway', 'Pakistan', 'Portugal', 'Russian Federation',
#                    'Saudi Arabia', 'Scotland', 'Singapore', 'South Korea', 'Spain',
#                    'Sweden', 'Thailand', 'Turkey', 'UK', 'USA']
#  ý tưởng tự động chạy 1 list hợp lệ các countries để check kh được , vì có khả năng 2 nước khác nhau nhưng độ fuzzy rất cao , làm replace sẽ nhầm, ví dụ như ireland-findland : 71, spain - japan : 60
# nên mình phải tự check tay từng từ và set min_ratio để tối ưu hóa

replace_matches_in_column(professors, "Country", "usa", min_ratio = 70)

# Check your answer
q3.check()

usa [('usa', 100), ('usofa', 75), ('austria', 60), ('australia', 50), ('spain', 50), ('urbana', 44), ('uk', 40), ('pakistan', 36), ('malaysia', 36), ('portugal', 36)]


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
# q3.hint()
# q3.solution()

# Congratulations!

Congratulations for completing the **Data Cleaning** course on Kaggle Learn!

To practice your new skills, you're encouraged to download and investigate some of [Kaggle's Datasets](https://www.kaggle.com/datasets).

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/data-cleaning/discussion) to chat with other learners.*